In [2]:
from pandas_summary import DataFrameSummary
import redcap
import pandas as pd
import macos_keychain
import re
import os

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [82]:
import logging
import logging.handlers
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG, format='[%(asctime)s]: %(levelname)s: %(message)s')

In [3]:
# Avoid plain text password (Reference: https://gist.github.com/gcollazo/9434580)
def getpassword(service, account):

    def decode_hex(s):
        s = eval('"' + re.sub(r"(..)", r"\x\1", s) + '"')
        if "" in s: s = s[:s.index("")]
        return s

    cmd = ' '.join([
        "/usr/bin/security",
        " find-generic-password",
        "-g -s '%s' -a '%s'" % (service, account),
        "2>&1 >/dev/null"
    ])
    p = os.popen(cmd)
    s = p.read()
    p.close()
    m = re.match(r"password: (?:0x([0-9A-F]+)\s*)?\"(.*)\"$", s)
    if m:
        hexform, stringform = m.groups()
        if hexform:
            return decode_hex(hexform)
        else:
            return stringform


def setpassword(service, account, password):
    cmd = 'security add-generic-password -U -a %s -s %s -p %s' % (account, service, password)
    p = os.popen(cmd)
    s = p.read()
    p.close()

In [54]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num, counter

In [4]:
KEYCHAIN = "UFHealth_mom_baby"

In [11]:
proj = redcap.Project('https://redcap.ctsi.ufl.edu/redcap/api/', getpassword(KEYCHAIN, "xinsongdu"))

In [17]:
proj.forms

('baby_hospital_dermatitis',
 'baby_hospital_sebaceous',
 'baby_hospital_erythema',
 'mom_antibiotics_ip',
 'baby_first_head_circumference',
 'baby_hospital_ear',
 'baby_wellvisit',
 'baby_clinic_erythema',
 'linked_mom_prenatal_abx_rx',
 'baby_hospital_asthma',
 'linked_mom_prental_meds_rx',
 'baby_hospital_obesity',
 'linked_mom_demography',
 'baby_clinic_ear',
 'baby_clinic_sebaceous',
 'baby_demography',
 'baby_hospital_hemangioma',
 'baby_antibiotics_rx',
 'mom_demography',
 'mom_medications_rx',
 'mom_antibiotics_rx',
 'baby_clinic_hemangioma',
 'mom_baby_link',
 'linked_mom_prenatal_apt',
 'baby_clinic_foodallergy',
 'baby_hospital_foodallergy',
 'baby_vaccines',
 'baby_clinic_eczema',
 'baby_clinic_dermatitis',
 'baby_clinic_obesity',
 'baby_clinic_asthma',
 'baby_antibiotics_ip',
 'mom_prenatal_apt',
 'linked_mom_antibiotics_ip',
 'baby_hospital_eczema',
 'baby_first_height')

In [163]:
def check_data(project_name, form_name, exl_file, exl_sheet, ret=False):
    data_redcap = project_name.export_records(forms=[form_name], format='df')
    data_redcap = data_redcap[data_redcap.redcap_repeat_instrument == form_name]
    logger.info("redcap data columns: %s", str(data_redcap.columns))
    logger.info("basic redcap data information:")
    logger.info("number of rows: %d; number of columns: %d; number of unique baby ids: %d; number of most frequently ids: %d", \
                data_redcap.shape[0], data_redcap.shape[1], \
                len(set(data_redcap[data_redcap.redcap_repeat_instrument==form_name].index.get_level_values('part_id'))), \
                most_frequent(list(data_redcap[data_redcap.redcap_repeat_instrument==form_name].index.get_level_values('part_id')))[1])
    # ICD codes in redcap data
    if len(list(data_redcap.select(lambda col: 'icd' in col or 'ICD' in col, axis=1)))!=0:
        unique_icd_redcap = sorted(data_redcap.select(lambda col: 'icd' in col or 'ICD' in col, axis=1).iloc[:,0].dropna().unique())
        logger.info("icd codes in redcap data:")
        logger.info(unique_icd_redcap)
    
    data_dropbox = pd.read_excel(io=exl_file, sheet_name=exl_sheet)
    logger.info("basic dropbox data information:")
    logger.info("number of rows: %d; number of columns: %d; number of unique baby ids: %d; number of most frequently ids: %d", \
            data_dropbox.shape[0], data_dropbox.shape[1], \
            len(set(data_dropbox.iloc[:,0])), \
            most_frequent(list(data_dropbox.iloc[:,0]))[1])    
    # ICD codes in dropbox data
    if len(list(data_dropbox.select(lambda col: 'icd' in col or 'ICD' in col, axis=1)))!=0:
        unique_icd_dropbox = sorted(data_dropbox.select(lambda col: 'icd' in col or 'ICD' in col, axis=1).iloc[:,0].dropna().unique())
        logger.info("icd codes in dropbox data:")
        logger.info(unique_icd_dropbox)

    if "unique_icd_redcap" in locals() and "unique_icd_dropbox" in locals():
        logger.info("icd code matched:")
        logger.info(unique_icd_dropbox == unique_icd_redcap)
        if not (unique_icd_dropbox == unique_icd_redcap):
            logger.info("icd codes in redcap data but not in dropbox data:")
            logger.info(set(unique_icd_redcap) - set(unique_icd_dropbox))
            logger.info("icd codes in dropbox data but not in redcap data:")
            logger.info(set(unique_icd_dropbox) - set(unique_icd_redcap))

    if ret == True:
        return data_redcap, data_dropbox


## baby_clinic_asthma

In [135]:
check_data(project_name=proj, form_name="baby_clinic_asthma", exl_file="../data/Baby-Billing Codes (Clinic).xlsx", exl_sheet="Asthma (Clinic)")

[2020-04-01 23:16:09,015]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-01 23:16:31,776]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-01 23:16:32,726]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance',
       'baby_asthma_date', 'days2_baby_asthma_clinic', 'baby_asthma_icd',
       'baby_clinic_asthma_complete'],
      dtype='object')
[2020-04-01 23:16:32,727]: INFO: basic redcap data information:
[2020-04-01 23:16:35,909]: INFO: number of rows: 13241; number of columns: 6; number of unique baby ids: 3577; number of most frequently ids: 75
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:9: Fu

## baby_hospital_asthma

In [149]:
check_data(project_name=proj, form_name="baby_hospital_asthma", exl_file="../data/Baby-Billing Codes (Hospital).xlsx", exl_sheet="Asthma")

[2020-04-01 23:36:35,526]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-01 23:37:03,286]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-01 23:37:03,633]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance',
       'baby_asthma_hosp_date', 'days2_baby_asthma_hosp',
       'baby_asthma_hosp_icd', 'baby_hospital_asthma_complete'],
      dtype='object')
[2020-04-01 23:37:03,634]: INFO: basic redcap data information:
[2020-04-01 23:37:04,144]: INFO: number of rows: 5293; number of columns: 6; number of unique baby ids: 2465; number of most frequently ids: 29
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher

## Baby Demography

In [150]:
check_data(project_name=proj, form_name="baby_demography", exl_file="../data/Baby.xlsx", exl_sheet="Baby")

[2020-04-01 23:43:06,865]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-01 23:43:26,588]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-01 23:43:27,550]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance', 'baby_race',
       'baby_ethnicity', 'baby_dob', 'baby_birth_wt_gr', 'delivery_mode',
       'baby_admit_date', 'baby_admit_source', 'baby_gest_age',
       'baby_nicu_los', 'days2_baby_admit', 'baby_gender', 'mom_id2',
       'mom_age_delivery', 'payer', 'baby_demography_complete'],
      dtype='object')
[2020-04-01 23:43:27,550]: INFO: basic redcap data information:
[2020-04-01 23:43:41,296]: INFO: number of rows: 30567; number of columns: 17; number of unique baby ids: 30567; number of most frequently ids: 1
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a fut

#### RedCap Data

In [19]:
data_redcap = proj.export_records(forms=['baby_demography'], format='df')

In [22]:
set(data_redcap.redcap_repeat_instrument)

{'baby_demography'}

In [23]:
data_redcap.shape

(30567, 17)

In [24]:
DataFrameSummary(data_redcap).summary()

,redcap_repeat_instrument,redcap_repeat_instance,baby_race,baby_ethnicity,baby_dob,baby_birth_wt_gr,delivery_mode,baby_admit_date,baby_admit_source,baby_gest_age,baby_nicu_los,days2_baby_admit,baby_gender,mom_id2,mom_age_delivery,payer,baby_demography_complete
count,NaN,30567,NaN,NaN,NaN,16681,NaN,NaN,NaN,NaN,3024,16684,NaN,NaN,16649,NaN,30567
mean,NaN,1,NaN,NaN,NaN,3045.43,NaN,NaN,NaN,NaN,12.5208,2.41531,NaN,NaN,27.6268,NaN,0
std,NaN,0,NaN,NaN,NaN,756.499,NaN,NaN,NaN,NaN,26.5958,310.917,NaN,NaN,5.95228,NaN,0
min,NaN,1,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,0,-5,NaN,NaN,12,NaN,0
25%,NaN,1,NaN,NaN,NaN,2720,NaN,NaN,NaN,NaN,1,0,NaN,NaN,23,NaN,0
50%,NaN,1,NaN,NaN,NaN,3158,NaN,NaN,NaN,NaN,3,0,NaN,NaN,27,NaN,0
75%,NaN,1,NaN,NaN,NaN,3523,NaN,NaN,NaN,NaN,9,0,NaN,NaN,32,NaN,0
max,NaN,1,NaN,NaN,NaN,6567,NaN,NaN,NaN,NaN,298,40160,NaN,NaN,56,NaN,0
counts,30567,30567,16684,16684,16684,16681,16684,16684,16684,15830,3024,16684,16649,16609,16649,16649,30567
uniques,1,1,10,5,2160,3148,13,2159,6,216,142,11,3,13836,39,11,1


In [41]:
data_redcap[pd.isnull(data_redcap.baby_race)]

,,redcap_repeat_instrument,redcap_repeat_instance,baby_race,baby_ethnicity,baby_dob,baby_birth_wt_gr,delivery_mode,baby_admit_date,baby_admit_source,baby_gest_age,baby_nicu_los,days2_baby_admit,baby_gender,mom_id2,mom_age_delivery,payer,baby_demography_complete
part_id,redcap_event_name,,,,,,,,,,,,,,,,,
Baby-16685,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,Mom-2033,31.0,Blue Cross,0
Baby-16686,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,Mom-8241,41.0,Medicaid HMO,0
Baby-16687,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,Mom-7413,29.0,Managed Care,0
Baby-16688,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,Mom-8556,26.0,Medicaid HMO,0
Baby-16689,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,Mom-13818,24.0,Medicaid,0
Baby-16690,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,NaN,26.0,Medicaid HMO,0
Baby-16691,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,Mom-13843,28.0,Medicaid HMO,0
Baby-16692,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,Mom-13282,27.0,Blue Cross,0
Baby-16693,visit_1_arm_1,baby_demography,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,Mom-13835,23.0,SELF PAY,0


16684 baby ids with demographic information, and the rest are mom-ids and baby ids with missing demographic information

#### Dropbox Data

In [27]:
data_dropbox = pd.read_excel(io="../data/Baby.xlsx", sheet_name="Baby")

In [28]:
data_dropbox.head()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS
0,Baby-0616,WHITE,NOT HISPANIC,1902-01-01,3627.0,"C-Section, Low Transverse",2011-12-15,INTRAMURAL BIRTH,39 4/7,NaN
1,Baby-0002,WHITE,NOT HISPANIC,2011-06-01,3417.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,38 3/7,NaN
2,Baby-0003,MULTIRACIAL,HISPANIC,2011-06-01,3383.0,"C-Section, Low Transverse",2011-06-01,INTRAMURAL BIRTH,40 4/7,NaN
3,Baby-0005,WHITE,NOT HISPANIC,2011-06-01,3882.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,NaN,NaN
4,Baby-0006,WHITE,NOT HISPANIC,2011-06-01,3710.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,39 6/7,NaN


In [29]:
data_dropbox.shape

(16684, 10)

In [31]:
DataFrameSummary(data_dropbox).summary()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS
count,NaN,NaN,NaN,NaN,16681,NaN,NaN,NaN,NaN,3024
mean,NaN,NaN,NaN,NaN,3045.43,NaN,NaN,NaN,NaN,12.5208
std,NaN,NaN,NaN,NaN,756.499,NaN,NaN,NaN,NaN,26.5958
min,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,0
25%,NaN,NaN,NaN,NaN,2720,NaN,NaN,NaN,NaN,1
50%,NaN,NaN,NaN,NaN,3158,NaN,NaN,NaN,NaN,3
75%,NaN,NaN,NaN,NaN,3523,NaN,NaN,NaN,NaN,9
max,NaN,NaN,NaN,NaN,6567,NaN,NaN,NaN,NaN,298
counts,16684,16684,16684,16684,16681,16684,16684,16684,15830,3024
uniques,16684,10,5,2160,3148,13,2159,6,220,142


## baby_first_height

In [151]:
check_data(project_name=proj, form_name="baby_first_height", exl_file="../data/Baby.xlsx", exl_sheet="First Height")

[2020-04-01 23:58:20,135]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-01 23:58:47,970]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-01 23:58:48,345]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance', 'baby_ht1_date',
       'days2_baby_ht1', 'baby_ht1_cm', 'baby_first_height_complete'],
      dtype='object')
[2020-04-01 23:58:48,346]: INFO: basic redcap data information:
[2020-04-01 23:58:52,773]: INFO: number of rows: 16441; number of columns: 6; number of unique baby ids: 16441; number of most frequently ids: 1
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
[2020-04-01 23:59:07,137]: INFO: basic dropbox data information:
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/s

## baby_first_head_circumference

In [152]:
check_data(project_name=proj, form_name="baby_first_head_circumference", exl_file="../data/Baby.xlsx", exl_sheet="First Head Circumference")

[2020-04-02 00:02:08,758]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-02 00:02:30,785]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-02 00:02:31,167]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance', 'baby_hc1_date',
       'days2_baby_hc1', 'baby_hc1_cm',
       'baby_first_head_circumference_complete'],
      dtype='object')
[2020-04-02 00:02:31,167]: INFO: basic redcap data information:
[2020-04-02 00:02:35,490]: INFO: number of rows: 16353; number of columns: 6; number of unique baby ids: 16346; number of most frequently ids: 2
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
[2020-04-02 00:02:49,985]: INFO: basic dropbox data information:
/Users/xinsongdu/.pyenv/versions/miniconda3-lat

## baby_antibiotics_rx

In [153]:
check_data(project_name=proj, form_name="baby_antibiotics_rx", exl_file="../data/Baby.xlsx", exl_sheet="Antibiotics IP Administration")

[2020-04-02 00:06:26,413]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-02 00:06:44,131]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-02 00:06:44,492]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance', 'baby_med_order',
       'baby_med_code', 'baby_meds', 'days2_baby_meds', 'baby_med_date',
       'baby_antibiotics_rx_complete'],
      dtype='object')
[2020-04-02 00:06:44,493]: INFO: basic redcap data information:
[2020-04-02 00:06:44,974]: INFO: number of rows: 5156; number of columns: 8; number of unique baby ids: 2717; number of most frequently ids: 25
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
[2020-04-02 00:07:00,590]: INFO: basic dropbox data information:
/Users/xinsongdu/.pyenv/

## baby_antibiotics_ip

In [154]:
check_data(project_name=proj, form_name="baby_antibiotics_ip", exl_file="../data/Baby.xlsx", exl_sheet="Antibiotics Prescription")

[2020-04-02 00:11:51,875]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-02 00:12:11,429]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-02 00:12:14,386]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance',
       'baby_med_order_ip', 'baby_mar_action_ip', 'baby_med_code_ip',
       'baby_med_ip', 'days2_baby_meds_ip', 'baby_med_ip_date',
       'baby_antibiotics_ip_complete'],
      dtype='object')
[2020-04-02 00:12:14,387]: INFO: basic redcap data information:
[2020-04-02 00:13:31,961]: INFO: number of rows: 70367; number of columns: 9; number of unique baby ids: 15998; number of most frequently ids: 537
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
[2020-04-02 00:13:46,235]: INFO: basic drop

## mom_antibiotics_ip

In [164]:
check_data(project_name=proj, form_name="mom_antibiotics_ip", exl_file="../data/Mom Medications With Added Pharm Classes.xlsx", exl_sheet="Mom Antibiotics IP Admin")

[2020-04-02 11:52:15,512]: DEBUG: Starting new HTTPS connection (1): redcap.ctsi.ufl.edu:443
[2020-04-02 11:52:41,400]: DEBUG: https://redcap.ctsi.ufl.edu:443 "POST /redcap/api/ HTTP/1.1" 200 None
[2020-04-02 11:52:42,130]: INFO: redcap data columns: Index(['redcap_repeat_instrument', 'redcap_repeat_instance',
       'mom_prenat_abxip_action', 'mom_prenat_abxip', 'mom_prenat_abxip_date',
       'mom_antibiotics_ip_complete'],
      dtype='object')
[2020-04-02 11:52:42,131]: INFO: basic redcap data information:
[2020-04-02 11:53:25,948]: INFO: number of rows: 51398; number of columns: 6; number of unique baby ids: 6740; number of most frequently ids: 574
/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
[2020-04-02 11:54:53,560]: INFO: basic dropbox data information:
[2020-04-02 11:55:39,054]: INFO: nu

## mom_antibiotics_rx

## mom_medications_ip

## mom_medications_rx

## mom_demography

## mom_prenatal_apt

## baby_clinic_foodallergy

## baby_clinic_ear

## baby_clinic_eczema

## baby_clinic_dermatitis

## baby_clinic_erythema

## baby_clinic_sebaceous

## baby_clinic_hemangioma

## baby_obesity_clinic

## baby_hospital_dermatitis

## baby_hospital_ear

## baby_hospital_eczema

## baby_hospital_foodallergy

## baby_hospital_hemangioma

## baby_hospital_sebaceous

## baby_hospital_obesity

## baby_hospital_erythema

## baby_vaccines

#### RedCap Data

In [43]:
data_redcap = proj.export_records(forms=['baby_vaccines'], format='df')

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [46]:
data_redcap.head()

redcap_repeat_instrument  redcap_repeat_instance  \
part_id   redcap_event_name                                                    
Baby-0001 visit_1_arm_1              baby_demography                       1   
Baby-0002 visit_1_arm_1              baby_demography                       1   
Baby-0003 visit_1_arm_1              baby_demography                       1   
          visit_1_arm_1                baby_vaccines                       1   
          visit_2_arm_1                baby_vaccines                       2   

                             days2_baby_vac baby_immune_date  \
part_id   redcap_event_name                                    
Baby-0001 visit_1_arm_1                 NaN              NaN   
Baby-0002 visit_1_arm_1                 NaN              NaN   
Baby-0003 visit_1_arm_1                 NaN              NaN   
          visit_1_arm_1                 1.0       2011-06-02   
          visit_2_arm_1                 2.0       2011-06-03   

                                                          baby_vac_name  \
part_id   redcap_event_name                                               
Baby-0001 visit_1_arm_1                                             NaN   
Baby-0002 visit_1_arm_1                                             NaN   
Baby-0003 visit_1_arm_1                                             NaN   
          visit_1_arm_1                         HEPATITIS_B_VACCINE&_IM   
          visit_2_arm_1      HEPATITIS_B_VACCINE_(PEDS/ADOL_3-DOSE)&_IM   

                             baby_vaccines_complete  
part_id   redcap_event_name                          
Baby-0001 visit_1_arm_1                         NaN  
Baby-0002 visit_1_arm_1                         NaN  
Baby-0003 visit_1_arm_1                         NaN  
          visit_1_arm_1                         0.0  
          visit_2_arm_1                         0.0

In [47]:
set(data_redcap.redcap_repeat_instrument)

{'baby_demography', 'baby_vaccines'}

In [49]:
data_redcap[data_redcap.redcap_repeat_instrument=="baby_vaccines"].shape

(105611, 6)

In [50]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument=="baby_vaccines"]).summary()

,redcap_repeat_instrument,redcap_repeat_instance,days2_baby_vac,baby_immune_date,baby_vac_name,baby_vaccines_complete
count,NaN,105611,95611,NaN,NaN,105611
mean,NaN,9.41286,270.59,NaN,NaN,0
std,NaN,6.49706,425.455,NaN,NaN,0
min,NaN,1,-5,NaN,NaN,0
25%,NaN,4,64,NaN,NaN,0
50%,NaN,9,151,NaN,NaN,0
75%,NaN,14,379,NaN,NaN,0
max,NaN,34,40222,NaN,NaN,0
counts,105611,105611,95611,105611,105611,105611
uniques,1,34,1739,2160,67,1


In [52]:
data_redcap.index.get_level_values('part_id')

Index(['Baby-0001', 'Baby-0002', 'Baby-0003', 'Baby-0003', 'Baby-0003',
       'Baby-0003', 'Baby-0003', 'Baby-0003', 'Baby-0003', 'Baby-0003',
       ...
       'Mom-13847', 'Mom-13848', 'Mom-13849', 'Mom-13850', 'Mom-13851',
       'Mom-13852', 'Mom-13853', 'Mom-13869', 'Mom-13874', 'Mom-13878'],
      dtype='object', name='part_id', length=136178)

In [56]:
most_frequent(list(data_redcap.index.get_level_values('part_id')))

('Baby-2489', 35)

In [58]:
len(set(data_redcap[data_redcap.redcap_repeat_instrument=="baby_vaccines"].index.get_level_values('part_id')))

13068

## baby_wellvisit

#### RedCap Data

In [61]:
data_redcap = proj.export_records(forms=['baby_wellvisit'], format='df')

In [62]:
data_redcap.head()

redcap_repeat_instrument  redcap_repeat_instance  \
part_id   redcap_event_name                                                    
Baby-0001 visit_1_arm_1              baby_demography                       1   
Baby-0002 visit_1_arm_1              baby_demography                       1   
Baby-0003 visit_1_arm_1              baby_demography                       1   
          visit_1_arm_1               baby_wellvisit                       1   
Baby-0004 visit_1_arm_1              baby_demography                       1   

                            baby_obs_date  baby_ht_cm  baby_wt_kgs  \
part_id   redcap_event_name                                          
Baby-0001 visit_1_arm_1               NaN         NaN          NaN   
Baby-0002 visit_1_arm_1               NaN         NaN          NaN   
Baby-0003 visit_1_arm_1               NaN         NaN          NaN   
          visit_1_arm_1        2014-06-30   99.799141    14.996886   
Baby-0004 visit_1_arm_1               NaN         NaN          NaN   

                             baby_head_circ_cm  days2_baby_wellvisit  \
part_id   redcap_event_name                                            
Baby-0001 visit_1_arm_1                    NaN                   NaN   
Baby-0002 visit_1_arm_1                    NaN                   NaN   
Baby-0003 visit_1_arm_1                    NaN                   NaN   
          visit_1_arm_1              51.104801                1125.0   
Baby-0004 visit_1_arm_1                    NaN                   NaN   

                             baby_wellvisit_complete  
part_id   redcap_event_name                           
Baby-0001 visit_1_arm_1                          NaN  
Baby-0002 visit_1_arm_1                          NaN  
Baby-0003 visit_1_arm_1                          NaN  
          visit_1_arm_1                          0.0  
Baby-0004 visit_1_arm_1                          NaN

In [63]:
set(data_redcap.redcap_repeat_instrument)

{'baby_demography', 'baby_wellvisit'}

In [66]:
data_redcap[data_redcap.redcap_repeat_instrument == "baby_wellvisit"].head()

redcap_repeat_instrument  redcap_repeat_instance  \
part_id   redcap_event_name                                                    
Baby-0003 visit_1_arm_1               baby_wellvisit                       1   
Baby-0004 visit_1_arm_1               baby_wellvisit                       1   
          visit_2_arm_1               baby_wellvisit                       2   
Baby-0009 visit_1_arm_1               baby_wellvisit                       1   
          visit_2_arm_1               baby_wellvisit                       2   

                            baby_obs_date  baby_ht_cm  baby_wt_kgs  \
part_id   redcap_event_name                                          
Baby-0003 visit_1_arm_1        2014-06-30   99.799141    14.996886   
Baby-0004 visit_1_arm_1        2011-06-07   48.498760     3.316892   
          visit_2_arm_1        2011-07-14   55.499001     4.847764   
Baby-0009 visit_1_arm_1        2011-06-06   51.998878     3.827182   
          visit_2_arm_1        2011-06-30   53.299358     4.762716   

                             baby_head_circ_cm  days2_baby_wellvisit  \
part_id   redcap_event_name                                            
Baby-0003 visit_1_arm_1              51.104801                1125.0   
Baby-0004 visit_1_arm_1              34.493198                   1.0   
          visit_2_arm_1              37.693600                  38.0   
Baby-0009 visit_1_arm_1              35.991798                   5.0   
          visit_2_arm_1              38.989002                  29.0   

                             baby_wellvisit_complete  
part_id   redcap_event_name                           
Baby-0003 visit_1_arm_1                          0.0  
Baby-0004 visit_1_arm_1                          0.0  
          visit_2_arm_1                          0.0  
Baby-0009 visit_1_arm_1                          0.0  
          visit_2_arm_1                          0.0

In [67]:
data_redcap[data_redcap.redcap_repeat_instrument == "baby_wellvisit"].shape

(51441, 8)

In [68]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument == "baby_wellvisit"]).summary()

,redcap_repeat_instrument,redcap_repeat_instance,baby_obs_date,baby_ht_cm,baby_wt_kgs,baby_head_circ_cm,days2_baby_wellvisit,baby_wellvisit_complete
count,NaN,51441,NaN,51441,51441,51441,51441,51441
mean,NaN,5.24403,NaN,64.91,7.04774,41.559,240.876,0
std,NaN,3.78735,NaN,13.283,3.40288,9.25128,261.836,0
min,NaN,1,NaN,17.5006,1.24738,2.99974,1,0
25%,NaN,2,NaN,53.2003,3.99728,36.8046,32,0
50%,NaN,4,NaN,63.5,6.66213,41.7068,140,0
75%,NaN,7,NaN,75.0011,9.49708,45.9994,373,0
max,NaN,43,NaN,168.148,73.9355,487.705,1816,0
counts,51441,51441,51441,51441,51441,51441,51441,51441
uniques,1,43,1670,1080,619,711,1353,1


In [69]:
len(set(data_redcap[data_redcap.redcap_repeat_instrument=="baby_wellvisit"].index.get_level_values('part_id')))

7667

In [70]:
most_frequent(list(data_redcap[data_redcap.redcap_repeat_instrument=="baby_wellvisit"].index.get_level_values('part_id')))

('Baby-12911', 43)

## baby_mom_baby_link

#### RedCap Data

In [71]:
data_redcap = proj.export_records(forms=['linked_mom_demography'], format='df')

In [72]:
data_redcap[data_redcap.redcap_repeat_instrument == 'linked_mom_demography'].head()

,,redcap_repeat_instrument,redcap_repeat_instance,mom_id,mom_race_link,mom_ethnicity_link,linked_mom_demography_complete
part_id,redcap_event_name,,,,,,
Baby-0001,visit_1_arm_1,linked_mom_demography,1,Mom-4388,WHITE,NOT HISPANIC,0.0
Baby-0002,visit_1_arm_1,linked_mom_demography,1,Mom-1829,WHITE,NOT HISPANIC,0.0
Baby-0003,visit_1_arm_1,linked_mom_demography,1,Mom-8494,OTHER,HISPANIC,0.0
Baby-0004,visit_1_arm_1,linked_mom_demography,1,Mom-0320,BLACK,NOT HISPANIC,0.0
Baby-0005,visit_1_arm_1,linked_mom_demography,1,Mom-6297,WHITE,NOT HISPANIC,0.0


In [74]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument == 'linked_mom_demography']).summary()

,redcap_repeat_instrument,redcap_repeat_instance,mom_id,mom_race_link,mom_ethnicity_link,linked_mom_demography_complete
count,NaN,16607,NaN,NaN,NaN,16607
mean,NaN,1.18793,NaN,NaN,NaN,0
std,NaN,0.448509,NaN,NaN,NaN,0
min,NaN,1,NaN,NaN,NaN,0
25%,NaN,1,NaN,NaN,NaN,0
50%,NaN,1,NaN,NaN,NaN,0
75%,NaN,1,NaN,NaN,NaN,0
max,NaN,5,NaN,NaN,NaN,0
counts,16607,16607,16607,16607,16607,16607
uniques,1,5,13844,11,4,1


In [79]:
data_redcap.shape

(47174, 6)

#### Dropbox Data

In [75]:
data_dropbox = pd.read_excel(io="../data/Baby.xlsx", sheet_name="Baby-Mom Link")

In [77]:
DataFrameSummary(data_dropbox).summary()

,Baby-Id,Mom-Id
count,29798,29793
unique,28069,27030
top,Mom-5930,Mom-1665
freq,5,5
counts,29798,29793
uniques,28069,27030
missing,0,5
missing_perc,0%,0.02%
types,categorical,categorical


## baby_first_height

#### RedCap Data

In [81]:
data_redcap = proj.export_records(forms=['baby_first_height'], format='df')

In [ ]:
data_redcap.head()

## linked_mom_demography

#### RedCap Data

In [9]:
data_redcap = proj.export_records(forms=['linked_mom_demography'], format='df')

In [14]:
data_redcap[data_redcap.redcap_repeat_instrument == 'linked_mom_demography'].head()

,,redcap_repeat_instrument,redcap_repeat_instance,mom_id,mom_race_link,mom_ethnicity_link,linked_mom_demography_complete
part_id,redcap_event_name,,,,,,
Baby-0001,visit_1_arm_1,linked_mom_demography,1,Mom-4388,WHITE,NOT HISPANIC,0.0
Baby-0002,visit_1_arm_1,linked_mom_demography,1,Mom-1829,WHITE,NOT HISPANIC,0.0
Baby-0003,visit_1_arm_1,linked_mom_demography,1,Mom-8494,OTHER,HISPANIC,0.0
Baby-0004,visit_1_arm_1,linked_mom_demography,1,Mom-0320,BLACK,NOT HISPANIC,0.0
Baby-0005,visit_1_arm_1,linked_mom_demography,1,Mom-6297,WHITE,NOT HISPANIC,0.0


In [15]:
data_redcap[data_redcap.redcap_repeat_instrument == 'linked_mom_demography'].shape

(16607, 6)

In [16]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument == 'linked_mom_demography']).summary()

,redcap_repeat_instrument,redcap_repeat_instance,mom_id,mom_race_link,mom_ethnicity_link,linked_mom_demography_complete
count,NaN,16607,NaN,NaN,NaN,16607
mean,NaN,1.18793,NaN,NaN,NaN,0
std,NaN,0.448509,NaN,NaN,NaN,0
min,NaN,1,NaN,NaN,NaN,0
25%,NaN,1,NaN,NaN,NaN,0
50%,NaN,1,NaN,NaN,NaN,0
75%,NaN,1,NaN,NaN,NaN,0
max,NaN,5,NaN,NaN,NaN,0
counts,16607,16607,16607,16607,16607,16607
uniques,1,5,13844,11,4,1


#### Dropbox Data

## mom-baby link

#### RedCap data

In [60]:
data_redcap = proj.export_records(forms=['mom_baby_link'], format='df')

In [69]:
data_redcap.head()

redcap_repeat_instrument  redcap_repeat_instance  \
part_id   redcap_event_name                                                    
Baby-0001 visit_1_arm_1              baby_demography                       1   
          visit_1_arm_1                mom_baby_link                       1   
Baby-0002 visit_1_arm_1              baby_demography                       1   
          visit_1_arm_1                mom_baby_link                       1   
Baby-0003 visit_1_arm_1              baby_demography                       1   

                             part_id2  mom_baby_link_complete  
part_id   redcap_event_name                                    
Baby-0001 visit_1_arm_1           NaN                     NaN  
          visit_1_arm_1           NaN                     0.0  
Baby-0002 visit_1_arm_1           NaN                     NaN  
          visit_1_arm_1           NaN                     0.0  
Baby-0003 visit_1_arm_1           NaN                     NaN

In [62]:
set(data_redcap["redcap_repeat_instrument"])

{'baby_demography', 'mom_baby_link'}

In [63]:
data_redcap[data_redcap.redcap_repeat_instrument=="mom_baby_link"].head()

,,redcap_repeat_instrument,redcap_repeat_instance,part_id2,mom_baby_link_complete
part_id,redcap_event_name,,,,
Baby-0001,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0002,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0003,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0004,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0005,visit_1_arm_1,mom_baby_link,1,NaN,0.0


In [64]:
data_redcap[data_redcap.redcap_repeat_instrument=="mom_baby_link"].shape

(29798, 4)

In [ ]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument=="baby_clinic_asthma"]).summary()

#### Dropbox data

In [65]:
data_dropbox = pd.read_excel(io="../data/Baby-Mom.xlsx", sheet_name="Baby-Mom Link")

In [66]:
data_dropbox.head()

,Baby-Id,Baby Gender,Mom-Id,Mom Age,Payer
0,Baby-0001,MALE,Mom-4388,25,Blue Cross
1,Baby-0002,FEMALE,Mom-1829,21,Medicaid
2,Baby-0003,MALE,Mom-8494,19,Medicaid
3,Baby-0004,FEMALE,Mom-0320,30,Medicaid
4,Baby-0005,FEMALE,Mom-6297,26,Medicaid


In [68]:
data_dropbox.shape

(16649, 5)

In [67]:
DataFrameSummary(data_dropbox).summary()

,Baby-Id,Baby Gender,Mom-Id,Mom Age,Payer
count,NaN,NaN,NaN,16649,NaN
mean,NaN,NaN,NaN,27.6268,NaN
std,NaN,NaN,NaN,5.95228,NaN
min,NaN,NaN,NaN,12,NaN
25%,NaN,NaN,NaN,23,NaN
50%,NaN,NaN,NaN,27,NaN
75%,NaN,NaN,NaN,32,NaN
max,NaN,NaN,NaN,56,NaN
counts,16649,16649,16609,16649,16649
uniques,16649,3,13836,39,11
